In [1]:
from collections import Counter
import pandas as pd
import re
from import_utils import utils

In [2]:
file = "entities/data/Inventare_st_michaelsburg_gesamt_korr.xlsx"

In [3]:
vbs = utils.get_filtered_sheets(file, filterstring='V')

In [4]:
dfs = utils.get_dfs(file, filterstring='V')

In [ ]:
# import easy values
# create baisc vocabs and relations
ww_kind, _ = WorkWorkRelation.objects.get_or_create(
    name='ist Teil von'
)
sla, _ = Institution.objects.get_or_create(
    name='Südtiroler Landesarchiv'
)
sla_vb_kind, _ = InstitutionWorkRelation.objects.get_or_create(
    name = 'archiviert'
)
sla_vb_kind.name_reverse = 'ist archiviert in'
sla_vb_kind.save()
for df in dfs:
    vb_name = df.loc[0][0].split(',')[0]
    vb_date = "{}".format([int(s) for s in vb_name.split() if s.isdigit()][0])
    vb, _ = Work.objects.get_or_create(
        name=vb_name
    )
    vb_kind, _ = WorkType.objects.get_or_create(
            name='Verfachbuch'
        )
    vb.kind = vb_kind
    vb.start_date_written = vb_date
    vb.end_date_written = "{}-12-31".format(vb_date)
    vb.save()
    sla_vb, _ = InstitutionWork.objects.get_or_create(
        relation_type=sla_vb_kind,
        related_institution=sla,
        related_work=vb
    )
    for i, row in df.iterrows():
        temp_work, _ = Work.objects.get_or_create(
            written_name=row['Signatur']
        )
        temp_work.name = row['Signatur']
        if row['Buch/Bücher'] == 'FALSE' or row['Buch/Bücher'] == 'n. a.':
            temp_work.buecher = ""
        else:
            temp_work.buecher = row['Buch/Bücher']
        if row['Summe Inventar'] == 'FALSE' or row['Summe Inventar'] == 'n. a.':
            temp_work.summe_inventar = ""
        else:
            temp_work.summe_inventar = row['Summe Inventar']
        if row['Barschaft (teilweise eigene Berechnung)'] == 'FALSE' or row['Barschaft (teilweise eigene Berechnung)'] == 'n.a.':
            temp_work.barschaft= ""
        else:
            temp_work.barschaft = row['Barschaft (teilweise eigene Berechnung)']
        temp_work.save()
    # kind
        raw_kind = row['Dokumententypus'].split('\n')[0].split(';')[0]
        work_kind, _ = WorkType.objects.get_or_create(
            name=raw_kind
        )
        temp_work.kind = work_kind
        temp_work.save()
    # link to vb
        temp_work_ww, _ = WorkWork.objects.get_or_create(
            relation_type=ww_kind,
            related_workA=temp_work,
            related_workB=vb
        )


In [ ]:
# add info about money, books, ...
temp_work = None
for df in dfs:
    for i, row in df.iterrows():
        temp_work = Work.objects.get(written_name=row['Signatur'])
        if row['Summe Inventar'] == 'FALSE' or row['Summe Inventar'] == 'n. a.':
            pass
        else:
            moneylist = [int(s) for s in row['Summe Inventar'].split() if s.isdigit()]
            if moneylist:
                try:
                    f, xr = moneylist[0], moneylist[1]
                except IndexError:
                    f, xr = moneylist[0], 0
            temp_work.invenatar_summe_norm = float("{}.{}".format(f, xr))
        temp_work.save()

In [12]:
# add Verstorbene and their Wohnorte
temp_work = None
place = None
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name='geführt als verstorben', name_reverse='führt als verstorben'
)
pers_place, _ = PersonPlaceRelation.objects.get_or_create(
    name='lebte in', name_reverse='Wohnort von'
)
for df in dfs:
    for i, row in df.iterrows():
        place = utils.create_places_from_place_string(
            row['Wohnort; Ort des betreffenden Gutes/der betreffenden Behausung'].split('\n')[0]
        )
        all_names = row['Name\n(Erklärung aus Verfachbuch)'].split('\n')[0]
        for written_name in all_names.split(';'):
            pers, _ = Person.objects.get_or_create(
                written_name=written_name,
                name=written_name
            )
            temp_work = Work.objects.get(written_name=row['Signatur'])
            pers_work_rel, _ = PersonWork.objects.get_or_create(
                relation_type=pers_work,
                related_person=pers,
                related_work=temp_work
            )
            pers_pl_rel, _ = PersonPlace.objects.get_or_create(
                relation_type=pers_place,
                related_person=pers,
                related_place=place[0]
            )            